# Project : Custom Cohere with memory

Have you may have seen, there is no memory conversation with our previous LLM. 

Chatbots are one of the central LLM use-cases. The core features of chatbots are that they can have long-running conversations and have access to information that users want to know about.

Aside from basic prompting and LLMs, memory and retrieval are the core components of a chatbot. Memory allows a chatbot to remember past interactions, and retrieval provides a chatbot with up-to-date, domain-specific information.

In this project we will focus on memory.

You will find everything needed here : https://python.langchain.com/docs/modules/memory/ 

### Complete pre-requisite

#### Packages

In [34]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain.schema import SystemMessage
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory,
                              ConversationBufferWindowMemory,
                              ConversationKGMemory)
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


# You can add other package here if needed
    

#### Initialize LLM & ConversationChain

A ConversationChain is a type of chain that is specifically designed to handle conversations or dialogues, as opposed to individual inputs.

In [35]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
COHERE_API_KEY='Cmey03cNZfTIxEf6anwTqB5THazodJSIgEKAE9yH'

# Initialisation de Cohere avec notre clé API
llm = Cohere(
            max_tokens=512, 
            cohere_api_key=COHERE_API_KEY)

conversation = ConversationChain(llm=llm)
print(llm)
#print(conversation.prompt.template)



Cohere
Params: {'model': None, 'max_tokens': 512, 'temperature': 0.75, 'k': 0, 'p': 1, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'truncate': None}


## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.



### ConversationBufferMemory

The conversation buffer memory keeps the previous pieces of conversation completely unmodified, in their raw form.

In [36]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation_buffer_memory = ConversationBufferMemory()

# Initialisation d'une ConversationChain avec le modèle de langage et la mémoire
conversation = ConversationChain(llm=llm, memory=conversation_buffer_memory)


#### verification

In [37]:
conversation.invoke("What is the capital of France ?")

conversation.invoke("What is the capital of Spain ?")

conversation.invoke("What is the capital of Angola ?")

# We should be able to get the whole conversation inside the memory
print(conversation.memory.buffer)

Human: What is the capital of France ?
AI:  The capital of France is Paris. It is located in the north-central part of the country, on the River Seine, and is one of the most populous and well-known cities in the world.

Paris is referred to as the City of Lights and the City of Love. It is home to some of the world's most iconic landmarks, including the Eiffel Tower, the Louvre Museum, the Arc de Triomphe, and Notre-Dame Cathedral. These monuments reflect Paris' rich history and cultural importance over the last millennia. 

Known for its artistic and intellectual heritage, Paris has been a major center for fashion, cuisine, and the arts for centuries. It's regarded as a leading global center for commerce, diplomacy, and education, hosting numerous prestigious universities and colleges, including the renowned Sorbonne.

Paris also hosts significant political institutions. It has been the center of French governance since the Middle Ages, with the Palais du Luxembourg serving as the se

#### From your observations - Gives the pros and cons of this memory 

##### Pros

- Contexte de court terme : Permet de maintenir un contexte de conversation à court terme en stockant les échanges récents.
- Rapidité : Peut améliorer la rapidité des réponses en réduisant la nécessité de retraiter des informations contextuelles à chaque interaction.

##### Cons

- Limitation de la mémoire : La taille du buffer est limitée, ce qui peut entraîner la perte d'informations contextuelles importantes pour des conversations très longues.
-



### ConversationSummaryMemory

The conversation summary memory creates a summary of the conversation over time. 

In [40]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

conversation_summary_memory = ConversationSummaryMemory(llm=llm)

# Initialisation d'une ConversationChain avec le modèle de langage et la mémoire de résumé
converstation_buffer_memory = ConversationChain(llm=llm, memory=conversation_summary_memory)

    

#### Verification

In [42]:
converstation_buffer_memory.invoke("Gives me the historical chronology of France. I want as much dates and events as possibles.")

converstation_buffer_memory.invoke("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

converstation_buffer_memory.invoke("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

print(converstation_buffer_memory.memory.buffer)

 The human asks the AI to provide a detailed historical chronology of Germany with dates and events. The AI provides a timeline of important events and dates spanning from 7000 BC to the present, covering various prehistoric cultures, the influence of the Roman Republic, the Germanic tribes, and medieval kingdoms. 


#### From your observations - Gives the pros and cons of this memory 

##### Pros

- utile pour les conversations longues où seul le contexte global est nécessaire
- aider le modèle de langage à produire des réponses cohérentes sans avoir à gérer un grand volume de texte

##### Cons

- Risque de perte d'information : Des résumés imprécis peuvent omettre des détails importants, ce qui pourrait nuire à la pertinence des réponses.
- Complexité de la génération de résumés : Générer des résumés précis et utiles est complexe et peut nécessiter des techniques avancées de NLP.



### ConversationSummaryBufferMemory

The conversation summary buffer memory keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions, it compiles them into a summary and uses both.

In [43]:

# Initialize a ConversationChain with a memory in it

# converstation_summary_buffer_memory = ConversationChain(....)

from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

conversation_summary_buffer_memory = ConversationSummaryBufferMemory(llm=llm)

# Initialisation d'une ConversationChain avec le modèle de langage et la mémoire spécifique
conversation = ConversationChain(llm=llm, memory=conversation_summary_buffer_memory)

#### Verification

In [44]:
conversation.invoke("Gives me the historical chronology of France. I want as much dates and events as possible.")

conversation.invoke("Gives me the historical chronology of France. I want as much dates and events as possibles.")

conversation.invoke("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

conversation.invoke("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

# We should be able to get the whole conversation inside the memory
print(conversation.memory.buffer)

[HumanMessage(content='Gives me the historical chronology of France. I want as much dates and events as possibles.'), AIMessage(content=" Certainly! Here's a comprehensive list of important dates and events in French history. \n\nFrench History: 481 AD to present times. \n\n1. The Kingdom of the Franks: 481 AD - 987 AD. Formation of the Kingdom of the Franks, with the merger of the Frankish and Merovingian tribes, establishing the first Christian kingdom in Europe under King Clovis I. Paris eventually becomes the capital. \n\n2. The Middle Ages: 481 AD - 14th century. French expansion and the Hundred Years' War (1337-1453) with England. War ended with the expulsion of English rule and the emergence of an absolute French monarchy, fostering a nascent French national identity and language. \n\n3. The Renaissance: 14th - 17th centuries. Becoming a major European power, with the French royal court as a cultural center fostering figures like Francois Rabelais, Montaigne, and Catherine de Me

#### From your observations - Gives the pros and cons of this memory 

##### Pros

- Ce type de mémoire permet un Équilibre entre mémoire tampon et résumé en combinant les avantages du buffering et du résumage.
- Flexibilité : Potentiellement ce type de mémoire est plus flexible dans la gestion des types de contexte de conversation.

##### Cons

- Complexité d'implémentation

